## Load Image

In [63]:
import os
import cv2

dataset_dir = 'small_dataset'

images = []
labels = []

for label_name in os.listdir(dataset_dir):
    label_path = os.path.join(dataset_dir, label_name)
    for filename in os.listdir(label_path):
        image_path = os.path.join(label_path, filename)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))
        images.append(image)
        labels.append(label_name)
print("load image done\n")

load image done



## Dataset

In [85]:

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

images = np.array(images)
labels = np.array(labels)

labels = LabelEncoder().fit_transform(labels)

x_train_val, x_test, y_train_val, y_test = train_test_split(images,labels,test_size=0.20)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

# Train

In [86]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(labels)), activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)


Epoch 1/10
3/3 [==============================] - 1s 269ms/step - loss: 1717.0897 - accuracy: 0.2500 - val_loss: 1636.7545 - val_accuracy: 0.1875
Epoch 2/10
3/3 [==============================] - 1s 233ms/step - loss: 979.2778 - accuracy: 0.2188 - val_loss: 102.0372 - val_accuracy: 0.2812
Epoch 3/10
3/3 [==============================] - 1s 213ms/step - loss: 154.2523 - accuracy: 0.2812 - val_loss: 11.4857 - val_accuracy: 0.4688
Epoch 4/10
3/3 [==============================] - 1s 211ms/step - loss: 36.8474 - accuracy: 0.3333 - val_loss: 8.5998 - val_accuracy: 0.5000
Epoch 5/10
3/3 [==============================] - 1s 215ms/step - loss: 7.4479 - accuracy: 0.5729 - val_loss: 4.4642 - val_accuracy: 0.5938
Epoch 6/10
3/3 [==============================] - 1s 205ms/step - loss: 2.6466 - accuracy: 0.7292 - val_loss: 2.6822 - val_accuracy: 0.5625
Epoch 7/10
3/3 [==============================] - 1s 205ms/step - loss: 0.8810 - accuracy: 0.7292 - val_loss: 2.3174 - val_accuracy: 0.5938
Epoch 

## Test

In [87]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

1/1 [==============================] - 0s 44ms/step - loss: 1.2607 - accuracy: 0.6875
Test Accuracy: 0.69


## Confution matrix

In [5]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

y_pred = model.predict(x_test)

y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# 3. 計算混淆矩陣
cm = confusion_matrix(y_true, y_pred_classes)

print(cm)
# 4. 畫圖
# plt.figure(figsize=(8,6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_names, yticklabels=label_names)
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix')
# plt.show()

[[14  3  0  0]
 [ 1 13  2  1]
 [ 2  5  8  0]
 [ 1  4  1  9]]
